In [43]:
import pandas as pd
import numpy as np

def map_feature_portfolio(infile, metric='%score', mean_across_maps=False, filters={'test_pos': 2}, unstack_level = 2):
    df = pd.read_csv(infile)
    df = df.rename(columns = {'strat':'portfolio'}) # renames the portfolio column

    # display(df.describe())
    
    # human-friendly portfolio names
    df.replace({'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush+suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush+defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'}, inplace=True)
    
    # the columns I want to see
    indexes = ['mapname', 'feature', 'portfolio', 'alpha', '_lambda', 'train_matches', 'dec_int', 'test_pos']
    
    #filters according to user criteria 
    for fname,fvalue in filters.items():
        if isinstance(fvalue, float):
            #print(fname, fvalue, df[fname])
            df = df[np.isclose(df[fname],fvalue)] 
        else:
            df = df[df[fname] == fvalue] 
        # drops the column
        df.drop(fname, axis=1, inplace=True)
        indexes.remove(fname)
        
    # display(df.describe())
    
    # builds a hierarchical (multiindex) data frame (kinda groups by map, strategy & units)
    hier = df.set_index(indexes) 
    #print(np.where(hier.index.duplicated()))  # finds entries with duplicate indexes, thx: https://stackoverflow.com/a/28652153/1251716
    
    # unstack the hierarchical dataframe up to the portfolio (all other parameters are grouped?)
    unstacked = hier.unstack(unstack_level)[metric]
    
    if mean_across_maps: 
        # if i group by the 'feature' level, i can get the mean on all maps played using that feature extractor
        return unstacked.groupby(level=['feature']).mean()
    else:
        return unstacked
        

In [ ]:
# displays the results for all adversaries
opponents = ['A3N', 'SSSmRTS','StrategyTactics','PuppetSearchMCTS','WorkerRush','LightRush'] 

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100, 'train_matches': 100000, '_lambda': 0.5, 'alpha': 0.01,}

for o in opponents:
    print("---{}, no search budget---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_avg.csv'.format(o), '%score', True, filters))
    
    print("---{}, search budget=100ms---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_b100_avg.csv'.format(o), '%score', True, filters))

In [ ]:
### ONLY LIGHT & WORKER RUSH WITH b0 and b100:
# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all'}

for o in ['WorkerRush', 'LightRush']:
    for b in [0, 100]:
        print("---{}, search budget={}ms---".format(o, b))
        display(map_feature_portfolio('../results/parameters/{}_b{}_avg.csv'.format(o,b), '%score', False, filters))

In [48]:
### This is to check the feature extractor

print('# Comparing feature extractors ')

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all',}# 'train_matches': 100000}

for o in ['WorkerRush', 'LightRush']:
    print("---{}---".format(o, b))
    display(map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o,b), '%score', False, filters, 1))
    
print('# Comparing feature extractors, now only at the end of training ')

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'train_matches': 100000}
for o in ['WorkerRush', 'LightRush']:
    print("---{}---".format(o, b))
    display(map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o,b), '%score', False, filters, 1))

# Comparing feature extractors 
---WorkerRush---


feature                           materialdistancehp  quadrantmodel
mapname            train_matches                                   
basesWorkers32x32A 0                       33.333333      66.666667
                   1000                    50.000000      50.000000
                   10000                   66.666667      66.666667
                   100000                  83.333333      50.000000
basesWorkers8x8    0                       16.666667       8.333333
                   1000                     0.000000      16.666667
                   10000                   25.000000      50.000000
                   100000                  58.333333      50.000000

---LightRush---


feature                           materialdistancehp  quadrantmodel
mapname            train_matches                                   
basesWorkers32x32A 0                       16.666667      33.333333
                   1000                    33.333333      33.333333
                   10000                   33.333333      33.333333
                   100000                  66.666667      16.666667
basesWorkers8x8    0                       50.000000      58.333333
                   1000                    50.000000      83.333333
                   10000                   66.666667     100.000000
                   100000                 100.000000     100.000000

# Comparing feature extractors, now only at the end of training 
---WorkerRush---


feature,materialdistancehp,quadrantmodel
mapname,,
basesWorkers32x32A,83.333333,50.0
basesWorkers8x8,58.333333,50.0


---LightRush---


feature,materialdistancehp,quadrantmodel
mapname,,
basesWorkers32x32A,66.666667,16.666667
basesWorkers8x8,100.000000,100.000000
